In [36]:
## fasta files used for this analysis are whole genome assemblies from 
## for custom db they need to be concatinated into one fasta file
## dowload fasta files and concatenate into one file
## examples
## for our test simulation I am using C.costata, D.mel, S.sech, D.sim, D.maur ( sech, sim, mar = close to mel ,cos = farthest)

#! mkdir -p ../sequences
#! cat ../sequences/D.mauritiana.fasta ../sequences/D.melanogaster.fasta ../sequences/D.sechellia.fasta ../sequences/D.simulans.fasta ../sequences/C.costata.fasta > ../sequences/combined_genomes.fasta
# for more sequences
# ! cat *fasta > many_seqs.fa
# use this in the folder where you have all of your fastas 

In [1]:
## make db will throw an error if any duplicate ids so we check for this
## some fasta files may still have duplicates and this will cause an error in make db, therefore we need to remove duplicates fasta files
## this can be done in 2 steps
## step 1) evaluate sequences/test_genomes_modified.fasta for duplicates and count if there are duplicates. 
## grep searches for > (start of fasta id line), uniq d prints out any duplicated non unique entries, wc -l counts the lines
! cat ../sequences/combined_genomes.fa | grep '^>' | sort | uniq -d | wc -l 

       0


In [4]:
## make db is particular about format of id so here we need to process the id's from our data to be compatable with blast makedb
## to parse ids for make db need 1) unique entries 2) names to not have blank spaces and other characters/less than 50 long
## for these files, we are keeping all alpha numberics and removing the end isolate number and linking names with "-"
## note I later needed to make a small python script to make all the sequences less than 50 characters long..
! sed -i 's/[^a-zA-Z0-9>_]/-/g; s/--organism//; s/---isolate//' ../sequences/combined_genomes.fasta > ../sequences/combined_genomes_modified_ids.fasta

In [14]:
## total sequences. this should be the number of sequences in output database. 
! cat ../sequences/combined_genomes_modified_ids.fasta | grep ">" | wc -l


cat: ../sequences/combined_genomes_modified_ids.fasta: No such file or directory
       0


In [16]:
# make database of sequences from concatinated sequence files (currently having issue bc won't parse ids correctly...)
# I removed the -parse_seqids and it made a list of files with db base name (need unique files for this )
# we will need to figure out why this is not working and add parse ids later 
! mkdir -p database
! makeblastdb -dbtype nucl -in ../sequences/analysis_files/combined_genomes_short_names  -out ../database/all_genomes_db -blastdb_version 5 -parse_seqids



Building a new DB, current time: 02/07/2021 15:43:01
New DB name:   /Users/giana.cirolia/enhancer_analysis_codebase/database/all_genomes_db
New DB title:  ../sequences/analysis_files/combined_genomes_short_names
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 97515 sequences in 249.802 seconds.




In [1]:
# make database of sequences from concatinated sequence files (currently having issue bc won't parse ids correctly...)
# I removed the -parse_seqids and it made a list of files with db base name (need unique files for this )
# we will need to figure out why this is not working and add parse ids later 
! mkdir -p database
! makeblastdb -dbtype nucl -in ../sequences/contigs_aligned_to_eve_gene_and_enhancer.fasta  -out ../database/whole_contig_db -blastdb_version 5 -parse_seqids




Building a new DB, current time: 02/04/2021 10:26:02
New DB name:   /Users/giana.cirolia/enhancer_analysis_codebase/database/whole_contig_db
New DB title:  ../sequences/contigs_aligned_to_eve_gene_and_enhancer.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 95 sequences in 21.2897 seconds.




In [4]:
! mkdir -p database
! makeblastdb -dbtype nucl -in ../sequences/eve_enhancer_extracted.fasta  -out ../database/larger_contigs_db -blastdb_version 5 -parse_seqids



Building a new DB, current time: 02/04/2021 11:08:34
New DB name:   /Users/giana.cirolia/enhancer_analysis_codebase/database/larger_contigs_db
New DB title:  ../sequences/eve_enhancer_extracted.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 73 sequences in 0.021543 seconds.




In [5]:
# check that your db created a database 
! blastdbcmd -db ../database/larger_contig_db -info

BLAST Database error: No alias or index file found for nucleotide database [../database/larger_contig_db] in search path [/Users/giana.cirolia/enhancer_analysis_codebase/notebooks::]


In [13]:
# count the number of sequence ids in your data base 
! blastdbcmd -db ../database/larger_contigs_db -entry all -outfmt %l

14783
13823
14784
14771
13804
13752
13758
14853
14730
13807
14746
14780
13835
13831
14780
13769
13834
14063
14548
13931
13781
14142
14462
13980
13806
13978
14622
14625
14339
14261
13996
14140
14488
14140
14367
14210
14395
14440
14141
14415
14415
14426
14168
14440
14266
14233
14334
14469
14430
14263
14266
14139
14139
14456
14416
14263
14263
14423
14266
14398
14138
14444
14337
14263
14263
14452
14464
14160
14337
14263
14263
14156
14182


In [38]:
! cat ../sequences/combined_genomes_short_names| grep ">" | awk -F"-" '{print$2"-"$3"-"$4"-"$5}' | sort | uniq -d | wc -l

^C


In [39]:
! blastdbcmd -db ../database/genomes_db -entry all -outfmt %i | awk -F'-' '{print$2"-"$3"-"$4"-"$5}'| sort | uniq -d > ../sequences/queries/species_in_db.txt

In [12]:
! cat ../sequences/combined_genomes_short_names | grep ">" | wc -l

   97515


In [11]:
## list of input species for comparison later
! blastdbcmd -db ../database/genomes_db -entry all -outfmt %i | awk -F"--" '{print $1}' | uniq -d > ../sequences/input_species.txt

In [10]:
! cat ../sequences/input_species.txt

In [ ]:
# if sequence ids equals input fastas you are correct. Here our sequence ids are not the names we want but its a start.

In [33]:
! sed 's/[^a-zA-Z0-9>_]/-/g; s/--organism//; s/---isolate//' ../sequences/mult_isolates/test.txt > test_changed.txt

In [2]:
## shorten name of sequencing files so are compatible with makdb 
with open("../sequences/combined_genomes_named.fa", "r") as fin:
    with open("../sequences/combined_genomes_short_names", "w") as fout:
        for line in fin:
            if line.startswith(">"):
                line_new = line.replace(line[:], line[0:49])
                fout.write("{}\n".format(line_new))
            else: 
                fout.write(line)

In [ ]:
! blastdbcmd -db ../database/genomes_db -entry all -outfmt %i